<a href="https://colab.research.google.com/github/nandhukumar86/AIML-Practice/blob/master/Pipleline_with_CrossVaidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline


In [2]:
df_train = pd.read_csv('/content/sample_data/california_housing_train.csv')
df_test = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [3]:
df_train.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [4]:
df_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [5]:
df_test.drop(columns= ['median_house_value'], inplace=True)

In [24]:
y_train = df_train.pop('median_house_value')

In [7]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,17000.0,-119.562108,2.005166,-124.3500,-121.790000,-118.4900,-118.000,-114.3100
latitude,17000.0,35.625225,2.137340,32.5400,33.930000,34.2500,37.720,41.9500
housing_median_age,17000.0,28.589353,12.586937,1.0000,18.000000,29.0000,37.000,52.0000
total_rooms,17000.0,2643.664412,2179.947071,2.0000,1462.000000,2127.0000,3151.250,37937.0000
total_bedrooms,17000.0,539.410824,421.499452,1.0000,297.000000,434.0000,648.250,6445.0000
population,17000.0,1429.573941,1147.852959,3.0000,790.000000,1167.0000,1721.000,35682.0000
households,17000.0,501.221941,384.520841,1.0000,282.000000,409.0000,605.250,6082.0000
median_income,17000.0,3.883578,1.908157,0.4999,2.566375,3.5446,4.767,15.0001
median_house_value,17000.0,207300.912353,115983.764387,14999.0000,119400.000000,180400.0000,265000.000,500001.0000


In [8]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,3000.0,-119.589200,1.994936,-124.1800,-121.810,-118.48500,-118.020000,-114.4900
latitude,3000.0,35.635390,2.129670,32.5600,33.930,34.27000,37.690000,41.9200
housing_median_age,3000.0,28.845333,12.555396,1.0000,18.000,29.00000,37.000000,52.0000
total_rooms,3000.0,2599.578667,2155.593332,6.0000,1401.000,2106.00000,3129.000000,30450.0000
total_bedrooms,3000.0,529.950667,415.654368,2.0000,291.000,437.00000,636.000000,5419.0000
population,3000.0,1402.798667,1030.543012,5.0000,780.000,1155.00000,1742.750000,11935.0000
households,3000.0,489.912000,365.422710,2.0000,273.000,409.50000,597.250000,4930.0000
median_income,3000.0,3.807272,1.854512,0.4999,2.544,3.48715,4.656475,15.0001


In [9]:
df_train.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
dtype: int64

In [10]:
df_test.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
dtype: int64

# Define the models in list to run inside a loop

In [81]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor

models = []
# models.append(['lr', LinearRegression()])
# models.append(['dtr', DecisionTreeRegressor()])
models.append(['knnr', KNeighborsRegressor()])


# Crossvalidation space should be defined for the pipeline

## Including knnr and dtr, mixing with others will start giving error stating the parameters does not belong to the model

## In short space should have parameters belonging to the input model.

In [82]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV 

space = dict()
space['knnr__n_neighbors'] = [5,10,15]
# space['knnr__weights'] = ['uniform', 'distance']
# space['dtr__criterion'] = ['mse','mae']
# space['dtr__splitter'] = ['best','random']
# space['dtr__max_depth'] = [5, 10, 15]
# space['dtr__min_samples_split'] = [5, 10, 15]
# space['dtr__min_samples_leaf'] = [1,3,5]

from sklearn.model_selection import KFold
cv_kfold = KFold(5)

# Create pipeline inside the loop, not outside.

In [85]:
for regressor in models:

  pl = Pipeline([
      ('scaler', StandardScaler()),
      ('pca', PCA(n_components = 0.95)),
      (regressor[0], regressor[1])
  ])

  model = GridSearchCV(pl, param_grid = space)

  model.fit(df_train, y_train)
  print('Training Score: ', model.score(df_train, y_train))
  print(model.best_score_)


Training Score:  0.6532667943671076
0.3234103574444795


In [86]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, df_train, y_train, cv = cv_kfold)
scores.mean()

# GridSearch CV score - reference sklearn API.
# model.best_score_


0.3234103574444795

### **cross_val_score** estimates the expected accuracy of your model on out-of-training data (pulled from the same underlying process as the training data, of course).